In [1]:
import numpy as np
import itertools
import math

In [9]:
def generate_line(d=2, seed=None):
    """
    Generates a random line y = w^T x + b in R^d (w, b ~ Normal(0,1)).
    Returns:
        w (ndarray): shape (d,), random normal
        b (float)  : random normal
    """
    if seed is not None:
        np.random.seed(seed)
    w = np.random.randn(d)
    b = np.random.randn()
    return w, b

def generate_points(w, b, n=20, seed=None):
    """
    Generates n random points x_i in R^d and corresponding y_i = w^T x_i + b.
    No additional noise is added here (but can be added if desired).
    Returns:
        X (ndarray): shape (n, d)
        y (ndarray): shape (n,)
    """
    d = len(w)
    if seed is not None:
        np.random.seed(seed)
    # For example, sample x_i from N(0, I)
    X = np.random.randn(n, d)
    # Compute y_i = w^T x_i + b
    y = X.dot(w) + b
    return X, y

def ridge_regression_closed_form(X, y, lam=1.0):
    """
    Fits a ridge regressor (L2-regularized least squares) of the form:
        theta = argmin_theta (||X*theta - y||^2 + lam * ||theta||^2)
    Here we handle the intercept by augmenting X with an all-ones column.
    Returns:
        theta (ndarray): shape (d+1,) the [w_est, b_est]
    """
    n, d = X.shape
    # Augment X with a column of 1s for the intercept
    X_aug = np.hstack([X, np.ones((n,1))])  # shape (n, d+1)

    # Solve (X_aug^T X_aug + lam I) theta = X_aug^T y
    A = X_aug.T.dot(X_aug) + lam * np.eye(d+1)
    b = X_aug.T.dot(y)
    theta = np.linalg.inv(A).dot(b)
    return theta

def compute_mse_full_dataset(theta, X, w_true, b_true):
    """
    Given learned parameters 'theta' (which is [w_est, b_est]) and the
    true line w_true, b_true, compute the MSE over the entire dataset X:

    MSE = (1/n) * sum_{i} ( (theta^T [x_i;1]) - (w_true^T x_i + b_true) )^2
    """
    n, d = X.shape
    w_est = theta[:d]
    b_est = theta[d]

    # True outputs
    y_true = X.dot(w_true) + b_true
    # Learned model outputs
    y_hat = X.dot(w_est) + b_est

    mse = np.mean((y_hat - y_true)**2)
    return mse

def greedy_subset_both_relevance_diversity(X, z, k, lam=1.0):
    """
    Implements the submodular greedy selection rule:
      x_i = argmax_{x in X\S_{i-1}}
             (z^T V_{S_{i-1}}^{-1} x)^2 / (1 + x^T V_{S_{i-1}}^{-1} x)
             + lam * log(1 + x^T V_{S_{i-1}}^{-1} x)
    where V_{S_{i-1}} = lam*I + sum_{x_j in S_{i-1}} x_j x_j^T.

    Inputs:
        X  (ndarray): shape (n, d)
        z  (ndarray): shape (d,) "test query" or reference vector
        k  (int)    : number of points to choose
        lam(float)  : regularization parameter for both V_S and the formula
    Returns:
        chosen_indices (list): The indices of X chosen by the submodular greedy method
    """
    n, d = X.shape

    chosen_indices = []
    # Start with V = lam * I
    V = lam * np.eye(d)

    # Keep track of which indices remain
    remaining = set(range(n))

    for _ in range(k):
        best_val = -np.inf
        best_idx = None

        # Invert V once per iteration
        V_inv = np.linalg.inv(V)

        for idx in remaining:
            x = X[idx]
            # Evaluate the objective
            # (z^T V^{-1} x)^2 / (1 + x^T V^{-1} x) + lam*log(1 + x^T V^{-1} x)
            numerator = (z @ V_inv @ x)**2
            denominator = 1.0 + x @ V_inv @ x
            val = (numerator / denominator) + lam * math.log(1.0 + x @ V_inv @ x)
            # val = (numerator / denominator)

            if val > best_val:
                best_val = val
                best_idx = idx

        # Add best_idx to chosen set
        chosen_indices.append(best_idx)
        remaining.remove(best_idx)

        # Update V = V + x best_idx x best_idx^T
        x_best = X[best_idx].reshape(d,1)
        V += x_best @ x_best.T

    return chosen_indices


In [10]:
def main_experiment():
    # 1) Generate the true line parameters w, b (dimension d=2 for illustration)
    d = 1
    w_true, b_true = generate_line(d=d, seed=42)

    # 2) Generate 20 random points from that line
    n = 30
    X, y = generate_points(w_true, b_true, n=n, seed=123)

    # 3) We set k=5
    k = 3

    # 4) Brute force search for the best subset of size k
    #    We fit a ridge regressor on each subset, then compute MSE w.r.t. the *true line*.
    #    We'll store the minimal MSE and the best subset of indices.
    from itertools import combinations

    best_mse = float('inf')
    best_subset = None

    all_indices = range(n)
    for subset_indices in combinations(all_indices, k):
        # Extract the subset
        X_sub = X[list(subset_indices)]
        y_sub = y[list(subset_indices)]

        # Fit ridge regression on these k points
        theta_sub = ridge_regression_closed_form(X_sub, y_sub, lam=1.0)

        # Compute MSE vs the true line w_true, b_true on all data X
        mse_val = compute_mse_full_dataset(theta_sub, X, w_true, b_true)

        if mse_val < best_mse:
            best_mse = mse_val
            best_subset = subset_indices

    # 5) Now use the submodular greedy approach
    #    We need a "test query" vector z. Let's just pick a random one from N(0,I).
    np.random.seed(999)
    z = np.random.randn(d)

    greedy_indices = greedy_subset_both_relevance_diversity(X, z, k=k, lam=1.0)

    # Fit ridge regression on the greedy subset
    X_greedy = X[greedy_indices]
    y_greedy = y[greedy_indices]
    theta_greedy = ridge_regression_closed_form(X_greedy, y_greedy, lam=1.0)
    greedy_mse = compute_mse_full_dataset(theta_greedy, X, w_true, b_true)

    # 6) Print results
    print("=== Experiment: Linear Model Subset Selection ===\n")
    # Some intermediate prints:
    print(f"1) Reference line (true) in R^{d}:")
    print(f"   w_true = {w_true}, b_true = {b_true}\n")

    print(f"2) The {n} points (X_i, y_i):")
    for i in range(n):
        print(f"   i={i}, X_i={X[i]}, y_i={y[i]:.4f}")
    print()

    print(f"3) Subset size k = {k}. Brute forcing all subsets to find best MSE...\n")

    print(f"4) Best subset found by brute force (indices) = {best_subset}")
    print(f"   Best subset MSE = {best_mse:.6f}\n")

    print("5) Submodular Greedy subset (indices) =", greedy_indices)
    print(f"   Greedy subset MSE = {greedy_mse:.6f}")

# Actually run the experiment
main_experiment()


=== Experiment: Linear Model Subset Selection ===

1) Reference line (true) in R^1:
   w_true = [0.49671415], b_true = -0.13826430117118466

2) The 30 points (X_i, y_i):
   i=0, X_i=[-1.0856306], y_i=-0.6775
   i=1, X_i=[0.99734545], y_i=0.3571
   i=2, X_i=[0.2829785], y_i=0.0023
   i=3, X_i=[-1.50629471], y_i=-0.8865
   i=4, X_i=[-0.57860025], y_i=-0.4257
   i=5, X_i=[1.65143654], y_i=0.6820
   i=6, X_i=[-2.42667924], y_i=-1.3436
   i=7, X_i=[-0.42891263], y_i=-0.3513
   i=8, X_i=[1.26593626], y_i=0.4905
   i=9, X_i=[-0.8667404], y_i=-0.5688
   i=10, X_i=[-0.67888615], y_i=-0.4755
   i=11, X_i=[-0.09470897], y_i=-0.1853
   i=12, X_i=[1.49138963], y_i=0.6025
   i=13, X_i=[-0.638902], y_i=-0.4556
   i=14, X_i=[-0.44398196], y_i=-0.3588
   i=15, X_i=[-0.43435128], y_i=-0.3540
   i=16, X_i=[2.20593008], y_i=0.9575
   i=17, X_i=[2.18678609], y_i=0.9479
   i=18, X_i=[1.0040539], y_i=0.3605
   i=19, X_i=[0.3861864], y_i=0.0536
   i=20, X_i=[0.73736858], y_i=0.2280
   i=21, X_i=[1.49073203], 